In [1]:
from flask import Flask , request ,jsonify
import numpy as np
import matplotlib.pyplot as plt
import numpy
import pickle
import os
#import cv2
import pandas as pd
#import mahotas as mt
from matplotlib import pyplot as plt

In [2]:
from PIL import Image

In [3]:
app = Flask(__name__)


def preprocess(image):
    names = ['area','perimeter','physiological_length','physiological_width','aspect_ratio','rectangularity','circularity', \
             'mean_r','mean_g','mean_b','stddev_r','stddev_g','stddev_b', \
             'contrast','correlation','inverse_difference_moments','entropy'
            ]
    
    main_img = image

    #Preprocessing
    img = cv2.cvtColor(main_img, cv2.COLOR_BGR2RGB)
    gs = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gs, (25,25),0)
    ret_otsu,im_bw_otsu = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    kernel = np.ones((50,50),np.uint8)
    closing = cv2.morphologyEx(im_bw_otsu, cv2.MORPH_CLOSE, kernel)

    #Shape features
    image, contours, _ = cv2.findContours(closing,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cnt = contours[0]
    M = cv2.moments(cnt)
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt,True)
    x,y,w,h = cv2.boundingRect(cnt)
    aspect_ratio = float(w)/h
    rectangularity = w*h/area
    circularity = ((perimeter)**2)/area

    #Color features
    red_channel = img[:,:,0]
    green_channel = img[:,:,1]
    blue_channel = img[:,:,2]
    blue_channel[blue_channel == 255] = 0
    green_channel[green_channel == 255] = 0
    red_channel[red_channel == 255] = 0

    red_mean = np.mean(red_channel)
    green_mean = np.mean(green_channel)
    blue_mean = np.mean(blue_channel)

    red_std = np.std(red_channel)
    green_std = np.std(green_channel)
    blue_std = np.std(blue_channel)

    #Texture features
    textures = mt.features.haralick(gs)
    ht_mean = textures.mean(axis=0)
    contrast = ht_mean[1]
    correlation = ht_mean[2]
    inverse_diff_moments = ht_mean[4]
    entropy = ht_mean[8]

    vector = [area,perimeter,w,h,aspect_ratio,rectangularity,circularity,\
              red_mean,green_mean,blue_mean,red_std,green_std,blue_std,\
              contrast,correlation,inverse_diff_moments,entropy
             ]

    df_temp = pd.DataFrame([vector],columns=names)
    
    print(df_temp)
    return df_temp

@app.route('/leaf' , methods = ['GET' , 'POST'])
def func():
    print("Upper")
    a=request.files['file']
    print("Lower")
    img = Image.open( a )
    I = numpy.asarray( img )
    plt.figure()
    plt.imshow( I )
    plt.show()
    
    output = preprocess(I)
    print(np.array(output))
    clf = pickle.load(open('svm_classifier.pkl' , 'rb')  )
    print(clf)
    prediction = clf.predict(np.array(output))
    myindex=prediction[0]
    print('predicted value : ' ,prediction)
    classes = ['Apple_Apple_scab',
           'Apple_Black_rot','Apple_healthy','Apple_Cedar_apple_rust','Blueberry_healthy','Cherry_healthy',
           'Cherry_Powdery_mildew','Corn_Cercospora_leaf_spot Gray_leaf_spot','Corn_Common_rust','Corn_Northern_Leaf_Blight',
           'Corn_healthy','Grape_Black_rot','Grape_Esca_(Black_Measles)','Grape_healthy','Grape_Leaf_blight',
           'Orange_Haunglongbing','Peach_Bacterial_spot','Pepper_bell_Bacterial_spot','Pepper_bell_healthy',
           'Peach_healthy','Potato_Early_blight','Potato_healthy','Potato_Late_blight','Raspberry_healthy',
           'Soybean_healthy','Squash_Powdery_mildew','Strawberry_healthy','Strawberry_Leaf_scorch',
           'Tomato_Bacterial_spot','Tomato_Tomato_Yellow_Leaf_Curl_Virus','Tomato_Tomato_mosaic_virus','Tomato_Target_Spot',
           'Tomato_Early_blight','Tomato_Spider_mites_Two-spotted_spider_mite','Tomato_Septoria_leaf_spot',
           'Tomato_Leaf_Mold','Tomato_Late_blight','Tomato_healthy']
    print(classes[myindex])
    return classes[myindex]

@app.route('/msp' , methods = ['GET' , 'POST'])
def func2():
    content=request.json
   # print(content["Crop"])
   # str=content["Crop"]
   # print (str)

    return "abcd"

In [5]:
#192.168.43.117
if __name__ == '__main__':
    app.run(host = "172.16.102.51" , port = 5001,debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://172.16.102.51:5001/ (Press CTRL+C to quit)
